In [1]:
!pip install mediapipe


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
from keras.models import load_model
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS ) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS , 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [5]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/content/data/MY_Data13') 

# Actions that we try to detect
actions = np.array(['hello', 
                    'Dhonnobad', 
                    'Dukkhito', 
                    'Apni', 
                    'Kemon', 
                    'Achen', 
                    'Ami', 
                    'Valo Achi', 
                    'Doya kore', 
                    'Aajker', 
                    'dekhte', 
                    'dinta', 
                    'Shundor', 
                    'Koita Baje', 
                    'Mach', 
                    'Shuvo Shokal', 
                    'SOS', 
                    '2', 
                    'Hata', 
                    'Fan', 
                    'Bati', 
                    'Ashirbad', 
                    'Bagh', 
                    'puru', 
                    'Shahajjo korte pari', 
                    'Boi', 
                    'mobile'])

label_map = {label:num for num, label in enumerate(actions)}

In [6]:
# load your model
model = load_model('LSTM_without_face_2.h5')

In [7]:
colors = [(16, 117, 245), (240, 98, 146), (46, 204, 113), (230, 126, 34), (31, 58, 147), 
          (241, 196, 15), (192, 57, 43), (52, 152, 219), (237, 76, 103), (39, 174, 96), 
          (247, 159, 31), (108, 122, 137), (211, 84, 0), (22, 160, 133), (243, 156, 18), 
          (149, 165, 166), (211, 84, 0), (46, 134, 193), (242, 38, 19), (52, 73, 94), 
          (200, 214, 229), (230, 126, 34), (155, 89, 182), (241, 196, 15), (16, 117, 245), 
          (17, 16, 245), (16, 117, 24)]
          
def prob_viz(res, actions, input_frame, colors):
    li = []
    for num, prob in enumerate(res):
        li.append((num, prob))
    sorted_list = sorted(li, key=lambda tup: tup[1], reverse=True)
    sorted_list = sorted_list[:5]
    i=0
    output_frame = input_frame.copy()
    for num, prob in sorted_list:
        cv2.rectangle(output_frame, (0,60+i*40), (int(prob*100), 90+i*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+i*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        i+=1
    
    return output_frame

In [8]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.1

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)    
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Boi
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Boi
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Boi
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Boi
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Boi
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'media

Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<cl

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Koita Baje
<class 'mediapipe.python.solution_base.SolutionO

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class '

Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<cl

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Aajker
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Aajker
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Aajker
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<c

<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Apni
<class '

KeyboardInterrupt: 

In [9]:
cap.release()
cv2.destroyAllWindows()